# Testing ```position_mask = packing_mask.all(1).all(1)```

In [14]:
import numpy as np
import torch

x = torch.tensor([[
                    [
                      [[ True,   False],
                      [ True,   True]],

                      [[ False,  False],
                        [ True,   False]],

                      [[ True,   False],
                        [ True,   True]],

                      [[ True,   False],
                        [ True,   True]],

                      [[ True,   False],
                        [ True,   True]],

                      [[ True,   False],
                        [ True,   True]]
                    ],

                    [
                      [[ True,   False],
                        [ True,   True]],

                      [[ True,   False],
                        [ True,   True]],

                      [[ True,   False],
                        [ True,   True]],

                      [[ True,   False],
                        [ True,   True]],

                      [[ True,   False],
                        [ True,   True]],

                        [[ True,   False],
                        [ True,   True]]
                      ]
                  ]])

print(f"Shape of x: {x.shape}")

x_flat = x.flatten(3, 4)
print(f"\n\nFlattened:\n{x_flat}\n")
print(f"Shape of x_flat: {x_flat.shape}")

x_all = x_flat.all(1)
print(f"\n\nx.all(1):\n{x_all}\n")
print(f"Shape of x_all: {x_all.shape}")


x_all_all = x_flat.all(1).all(1)
print(f"\n\nx.all(1).all(1):\n{x_all_all}\n")
print(f"Shape of x_all_all: {x_all_all.shape}")

x_premuted = x_flat.permute(0, 3, 1, 2)
print(f"\n\nx_flat.permuted\n{x_premuted}\n")
print(f"Shape of x_flat.permuted: {x_premuted.shape}")


Shape of x: torch.Size([1, 2, 6, 2, 2])


Flattened:
tensor([[[[ True, False,  True,  True],
          [False, False,  True, False],
          [ True, False,  True,  True],
          [ True, False,  True,  True],
          [ True, False,  True,  True],
          [ True, False,  True,  True]],

         [[ True, False,  True,  True],
          [ True, False,  True,  True],
          [ True, False,  True,  True],
          [ True, False,  True,  True],
          [ True, False,  True,  True],
          [ True, False,  True,  True]]]])

Shape of x_flat: torch.Size([1, 2, 6, 4])


x.all(1):
tensor([[[ True, False,  True,  True],
         [False, False,  True, False],
         [ True, False,  True,  True],
         [ True, False,  True,  True],
         [ True, False,  True,  True],
         [ True, False,  True,  True]]])

Shape of x_all: torch.Size([1, 6, 4])


x.all(1).all(1):
tensor([[False, False,  True, False]])

Shape of x_all_all: torch.Size([1, 4])


x_flat.permuted
tensor([[[[ True

In [1]:
import plotly.graph_objects as go
import random
import numpy as np

def cube_trace(x, y, z, length, width, height, scale, colour):
    edges = go.Scatter3d(                                                                                       # Some edges have to be drawn twice at a cube is no Euler circle or Euler path
                                                                                                                # (x[i]y[i]z[i]) is a coordinate, (x[i+1]y[i+1]z[i+1]) the next. A line is drawn between them and so on, making the edges of the cuboids
        x = [x, x + length, x + length, x,         x, x,          x + length, x + length, x,          x,          x + length, x + length, x + length, x + length, x,          x        ],
        y = [y, y,          y + width,  y + width, y, y,          y,          y + width,  y + width,  y,          y,          y,          y + width,  y + width,  y + width,  y + width],
        z = [z, z,          z,          z,         z, z + height, z + height, z + height, z + height, z + height, z + height, z,          z,          z + height, z + height, z        ],

        marker = dict(size = 1),
        line = dict(color = 'black', width = 3)
    )

    x      +=     scale                                                                                         # Small inward shift by scale so that the edges remain visible without overlapping the mesh Reduces the dimensions accordingly (dx -= 2*scale, etc.) Purpose: Edges are not covered by the mesh surface, resulting in a cleaner visual appearance
    y      +=     scale
    z      +=     scale
    length -= 2 * scale
    width  -= 2 * scale
    height -= 2 * scale

    colour = 'rgb(%s,%s,%s)' % colour
    
    surface = go.Mesh3d(
        x = [x, x,         x + length, x + length, x,          x,          x + length, x + length],             # Coordinates of the corner points of the cuboid
        y = [y, y + width, y + width,  y,          y,          y + width,  y + width,  y         ],
        z = [z, z,         z,          z,          z + height, z + height, z + height, z + height],

        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],                                                               # Mesh3d draws triangles. 2 triangles = 1 cuboid
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],                                                               # Those are the indices of the corner points (x[i], y[i], z[i])
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],                                                               # (i[i]j[i]k[i]) and (i[i+1]j[i+1]k[i+1]) together form one of the 6 surfaces of each cuboid. (i[i+2]j[i+2]k[i+2]) and (i[i+3]j[i+3]k[i+3]) together the next one and so on
        opacity = 1,
        color = colour,
        flatshading = True
    )

    return edges, surface


def plot_result(packing_result, bin_size_x, bin_size_y):
    '''
    packing_result: [box1, box2, box3, ...]
    box1:[length, width, height, x, y, z]
    '''

    # Check, whetcher boxes are overlapping
    box_num = len(packing_result)
    for i in range(box_num - 1):
        for j in range(i + 1, box_num):
            box_i = np.array(packing_result[i])
            box_j = np.array(packing_result[j])
            box_i_centre = box_i[0:3] / 2 + box_i[3:]
            box_j_centre = box_j[0:3] / 2 + box_j[3:]
            is_overlap = (np.abs(box_j_centre - box_i_centre) < (box_i[0:3] + box_j[0:3]) / 2).all()
            if is_overlap:
                raise Exception(
                        f"Items are overlapping. Please check the outcome.\n"
                        f"Overlapping boxes:\n"
                        f"Box with coordinates({box_i[3]}, {box_i[4]}, {box_i[5]}), with size ({box_i[0]}, {box_i[1]}, {box_i[2]}) and\n"
                        f"Box with coordinates({box_j[3]}, {box_j[4]}, {box_j[5]}), with size ({box_j[0]}, {box_j[1]}, {box_j[2]})"
                    )

    # Calculate the use_ratio
    packing_array = np.array(packing_result)
    box_height_coordinate = packing_array[:, 2] + packing_array[:, 5]
    max_height = np.max(box_height_coordinate)
    use_ratio = packing_array[:, :3].prod(1).sum() / (bin_size_x * bin_size_y * max_height)
    print("[The use ratio is: %.2f %%]"%(use_ratio * 100))

    # Plot result
    traces = []
    colour_limit_min = 0
    colour_limit_max = 255
    for box in packing_result:
        colour = (random.randint(colour_limit_min, colour_limit_max),
                  random.randint(colour_limit_min, colour_limit_max),
                  random.randint(colour_limit_min, colour_limit_max)
                )
        length, width, height, x, y, z = box
        scale = 0.0007 * max(bin_size_x, bin_size_y)
        box_edges, box_surfaces = cube_trace(x, y, z, length, width, height, scale, colour)
        traces.append(box_surfaces)
        traces.append(box_edges)

    container_edges, _  = cube_trace(0, 0, 0, bin_size_x, bin_size_y, max_height, scale, (255, 255, 255))
    traces.append(container_edges)

    figure = go.Figure(data = traces)

    figure.update_layout(scene = dict(
        xaxis = dict(showbackground = False, color = 'white'),
        yaxis = dict(showbackground = False, color = 'white'),
        zaxis = dict(showbackground = False, color = 'white'),
        aspectmode = 'manual',
        aspectratio = dict(x = bin_size_x,
                           y = bin_size_y,
                           z = max_height
                        )     
                    ),
                    showlegend = False
                )
   
    figure.show()


bin_size_x = bin_size_y = 10
packing_result = [[2, 2, 2, 0, 0, 0],
                  [4, 6, 3, 2, 0, 0],  
                  [2, 4, 2, 0, 2, 0],
                  [3, 3, 5, 7, 0, 0],
                  [7, 3, 2, 0, 0, 3],
                  [4, 3, 5, 6, 3, 0],
                  [5, 4, 6, 0, 6, 0],
                  [2, 3, 3, 0, 3, 2],
                  [4, 3, 2, 2, 3, 3],
                  [10, 5, 5, 0, 0, 5],
                  [4, 4, 5, 5, 6, 0],
                  [5, 5, 5, 5, 5, 5],
                  [5, 5, 4, 0, 5, 6]
                ]

plot_result(packing_result, bin_size_x, bin_size_y)

[The use ratio is: 96.00 %]
